In [ ]:
! pip install datasets
! pip install transformers

     |████████████████████████████████| 312 kB 7.9 MB/s 
     |████████████████████████████████| 243 kB 39.4 MB/s 
     |████████████████████████████████| 1.1 MB 10.7 MB/s 
     |████████████████████████████████| 133 kB 55.9 MB/s 
     |████████████████████████████████| 67 kB 3.0 MB/s 
     |████████████████████████████████| 144 kB 45.4 MB/s 
     |████████████████████████████████| 271 kB 59.1 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 3.5 MB 6.5 MB/s 
     |████████████████████████████████| 6.8 MB 36.5 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 596 kB 59.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import random
import numpy as np
import pandas as pd
from ast import literal_eval as load
from datasets import Dataset
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers.models.bert.modeling_bert import BertForTokenClassification
from transformers import BertTokenizer, BertConfig, AdamW, get_cosine_schedule_with_warmup
from ast import literal_eval as load


In [ ]:
from google.colab import drive 
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
model_path = "/content/gdrive/My Drive/scibert"
tokenizer = BertTokenizer.from_pretrained(model_path)
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")


In [ ]:
# BIO is not required.

def to_BIO(label_seqs):
  new_label_segs = []
  prev = ""
  for label in label_seqs:
    if label == prev:
      new_label = "I_" + label
    else:
      new_label = "B_" + label
    prev = label
    new_label_segs.append(new_label)
  return new_label_segs

In [ ]:
maxlen = 512
pad_token_id = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
ignore_label_id = CrossEntropyLoss().ignore_index


def convert(df):
    input_ids_ls = []
    attention_mask_ls = []
    labels_ls = []
    for i in range(len(df)):
        sents = df.loc[i, 'text']
        # labels = df.loc[i, 'labels']
        labels = df.loc[i, 'labels_code']
        tokens = []
        label_ids = []
        for j in range(len(sents)):  # loop over each sentence
            token_tmp = []
            words = tokenizer.tokenize(sents[j]) 
            words = list(map(lambda x: x.lower(), words))
            token_tmp.extend(words)
            token_tmp.extend([tokenizer.sep_token])
            label_ids.extend([ignore_label_id] *
                                (len(token_tmp)-1)+[labels[j]])
            tokens.extend(token_tmp)
        if len(tokens) > maxlen:
            tokens = tokens[:maxlen]
            label_ids = label_ids[:maxlen]
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        padding_length = maxlen - len(input_ids)
        attention_mask = [1]*len(input_ids) + [0]*padding_length
        input_ids.extend([pad_token_id] * padding_length)
        label_ids.extend([ignore_label_id] * padding_length)
        assert len(input_ids) == maxlen
        assert len(attention_mask) == maxlen
        assert len(label_ids) == maxlen
        input_ids_ls.append(input_ids)
        attention_mask_ls.append(attention_mask)
        labels_ls.append(label_ids)
    tokenized_df = pd.DataFrame(
        [input_ids_ls, attention_mask_ls, labels_ls]).transpose()
    tokenized_df.columns = ['input_ids',
                            'attention_mask', 'labels']
    return tokenized_df

In [ ]:
dow = {
    "BACKGROUND": 0,
    "OBJECTIVE": 1,
    "METHODS": 2,
    "RESULTS": 3, 
    "CONCLUSIONS": 4
}

In [ ]:
train_x_single = []
train_y_single = []
f = open("/content/gdrive/My Drive/manually_annotated_data/manually_annotated_train.txt", "r")
for x in f:
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
# digits were eliminated

        abstract_ = splitted[1].strip('\n')
        abstract_result = ''.join(i for i in abstract_ if not i.isdigit())
        train_x_single.append(abstract_result)
        label_ = splitted[0].strip('\n')
        train_y_single.append(label_)

f.close()

test_x_single = []
test_y_single = []
f = open("/content/gdrive/My Drive/manually_annotated_data/manually_annotated_test.txt", "r")
for x in f:
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
# digits were eliminated

        abstract_ = splitted[1].strip('\n')
        abstract_result = ''.join(i for i in abstract_ if not i.isdigit())
        test_x_single.append(abstract_result)
        label_ = splitted[0].strip('\n')
        test_y_single.append(label_)

f.close()


In [ ]:
manually_annotated_train_x_sequence = []
manually_annotated_train_y_sequence = []
manually_annotated_dev_x_sequence = []
manually_annotated_dev_y_sequence = []

import re

abstract = []
label = []
PMID = []
f = open("/content/gdrive/My Drive/manually_annotated_data/manually_annotated_train.txt", "r")
for x in f:
    if x[:3] == "###":
        if abstract != []:
            manually_annotated_train_x_sequence.append(abstract)
            # train_y_sequence.append([dow[x] for x in label])
            manually_annotated_train_y_sequence.append(label)
            PMID.append(x[3:])
        abstract = []
        label = []
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
        abstract_ = splitted[1].strip('\n')
        abstract_result = re.sub("\d+", "@", abstract_)
        abstract.append(abstract_result)
        label_ = splitted[0].strip('\n')
        label.append(label_)

manually_annotated_train_x_sequence.append(abstract)
manually_annotated_train_y_sequence.append(label)

f.close()

abstract = []
label = []

f = open("/content/gdrive/My Drive/manually_annotated_data/manually_annotated_test.txt", "r")
for x in f:
    if x[:3] == "###":
        if abstract != []:
            manually_annotated_dev_x_sequence.append(abstract)
            # dev_y_sequence.append([dow[x] for x in label])
            manually_annotated_dev_y_sequence.append(label)
        abstract = []
        label = []
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
        abstract_ = splitted[1].strip('\n')
        abstract_result = ''.join(i for i in abstract_ if not i.isdigit())
        abstract.append(abstract_result)
        label_ = splitted[0].strip('\n')
        label.append(label_)
manually_annotated_dev_x_sequence.append(abstract)
manually_annotated_dev_y_sequence.append(label)
f.close()

In [ ]:
manually_annotated_dev_x_sequence_structured = []
manually_annotated_dev_y_sequence_structured = []

abstract = []
label = []

f = open("/content/gdrive/My Drive/manually_annotated_data/manually_annotated_structured_test.txt", "r")
for x in f:
    if x[:3] == "###":
        if abstract != []:
            manually_annotated_dev_x_sequence_structured.append(abstract)
            # dev_y_sequence.append([dow[x] for x in label])
            manually_annotated_dev_y_sequence_structured.append(label)
        abstract = []
        label = []
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
        abstract_ = splitted[1].strip('\n')
        abstract_result = ''.join(i for i in abstract_ if not i.isdigit())
        abstract.append(abstract_result)
        label_ = splitted[0].strip('\n')
        label.append(label_)
manually_annotated_dev_x_sequence_structured.append(abstract)
manually_annotated_dev_y_sequence_structured.append(label)
f.close()

In [ ]:
manually_annotated_dev_x_sequence_unstructured = []
manually_annotated_dev_y_sequence_unstructured = []

abstract = []
label = []

f = open("/content/gdrive/My Drive/manually_annotated_data/manually_annotated_unstructured_test.txt", "r")
for x in f:
    if x[:3] == "###":
        if abstract != []:
            manually_annotated_dev_x_sequence_unstructured.append(abstract)
            # dev_y_sequence.append([dow[x] for x in label])
            manually_annotated_dev_y_sequence_unstructured.append(label)
        abstract = []
        label = []
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
        abstract_ = splitted[1].strip('\n')
        abstract_result = ''.join(i for i in abstract_ if not i.isdigit())
        abstract.append(abstract_result)
        label_ = splitted[0].strip('\n')
        label.append(label_)
manually_annotated_dev_x_sequence_unstructured.append(abstract)
manually_annotated_dev_y_sequence_unstructured.append(label)
f.close()

In [ ]:
pubmed_test_x_sequence = []
pubmed_test_y_sequence = []

abstract = []
label = []

f = open("/content/gdrive/My Drive/imrd_data_pubmed/test.txt", "r")
for x in f:
    if x[:3] == "###":
        if abstract != []:
            pubmed_test_x_sequence.append(abstract)
            # dev_y_sequence.append([dow[x] for x in label])
            pubmed_test_y_sequence.append(label)
        abstract = []
        label = []
    if x[:3] != "###" and x != "\n":
        splitted = x.split('\t')
        if len(splitted) < 2:
            continue
        abstract_ = splitted[1].strip('\n')
        abstract_result = ''.join(i for i in abstract_ if not i.isdigit())
        abstract.append(abstract_result)
        label_ = splitted[0].strip('\n')
        label.append(label_)
pubmed_test_x_sequence.append(abstract)
pubmed_test_y_sequence.append(label)
f.close()

In [ ]:
manually_annotated_train_df = pd.DataFrame({"text": manually_annotated_train_x_sequence, "labels": manually_annotated_train_y_sequence})
manually_annotated_eval_df = pd.DataFrame({"text": manually_annotated_dev_x_sequence, "labels": manually_annotated_dev_y_sequence})
manually_annotated_eval_df_structured = pd.DataFrame({"text": manually_annotated_dev_x_sequence_structured, "labels": manually_annotated_dev_y_sequence_structured})
manually_annotated_eval_df_unstructured = pd.DataFrame({"text": manually_annotated_dev_x_sequence_unstructured, "labels": manually_annotated_dev_y_sequence_unstructured})
pubmed_eval_df = pd.DataFrame({"text": pubmed_test_x_sequence, "labels": pubmed_test_y_sequence})

In [ ]:
l_text = []
l_label = []
for key, i in manually_annotated_train_df.iterrows():
  for j in range(len(i["labels"]) - 2):
    l_text.append(i["text"][j:j+3])
    l_label.append(i["labels"][j:j+3])
df_three_window_train = pd.DataFrame(list(zip(l_text, l_label)), columns =['text', 'labels'])

In [ ]:
l_text = []
l_label = []
for key, i in manually_annotated_eval_df.iterrows():
  for j in range(len(i["labels"]) - 2):
    l_text.append(i["text"][j:j+3])
    l_label.append(i["labels"][j:j+3])
df_three_window_test = pd.DataFrame(list(zip(l_text, l_label)), columns =['text', 'labels'])

In [ ]:
l_text = []
l_label = []
for key, i in manually_annotated_eval_df_structured.iterrows():
  for j in range(len(i["labels"]) - 2):
    l_text.append(i["text"][j:j+3])
    l_label.append(i["labels"][j:j+3])
df_three_window_test_structured = pd.DataFrame(list(zip(l_text, l_label)), columns =['text', 'labels'])

In [ ]:
l_text = []
l_label = []
for key, i in manually_annotated_eval_df_unstructured.iterrows():
  for j in range(len(i["labels"]) - 2):
    l_text.append(i["text"][j:j+3])
    l_label.append(i["labels"][j:j+3])
df_three_window_test_unstructured = pd.DataFrame(list(zip(l_text, l_label)), columns =['text', 'labels'])

In [ ]:
l_text = []
l_label = []
for key, i in pubmed_eval_df.iterrows():
  for j in range(len(i["labels"]) - 2):
    l_text.append(i["text"][j:j+3])
    l_label.append(i["labels"][j:j+3])
df_three_window_test_pubmed = pd.DataFrame(list(zip(l_text, l_label)), columns =['text', 'labels'])

In [ ]:
test_y = [[dow[j] for j in i] for i in manually_annotated_dev_y_sequence]
test_y_structured = [[dow[j] for j in i] for i in manually_annotated_dev_y_sequence_structured]
test_y_unstructured = [[dow[j] for j in i] for i in manually_annotated_dev_y_sequence_unstructured]
test_y_pubmed = [[dow[j] for j in i] for i in pubmed_test_y_sequence]

In [ ]:
df_three_window_train["labels_BIO"] = df_three_window_train["labels"].apply(to_BIO)
df_three_window_test["labels_BIO"] = df_three_window_test["labels"].apply(to_BIO)
df_three_window_test_structured["labels_BIO"] = df_three_window_test_structured["labels"].apply(to_BIO)
df_three_window_test_unstructured["labels_BIO"] = df_three_window_test_unstructured["labels"].apply(to_BIO)
df_three_window_test_pubmed["labels_BIO"] = df_three_window_test_pubmed["labels"].apply(to_BIO)

def rep_BIO(seq_label):
  dow_with_BIO = {
    "B_BACKGROUND": 0,
    "I_BACKGROUND": 1,
    "B_OBJECTIVE": 2,
    "I_OBJECTIVE": 3,
    "B_METHODS": 4,
    "I_METHODS": 5,
    "B_RESULTS": 6, 
    "I_RESULTS": 7,
    "B_CONCLUSIONS": 8,
    "I_CONCLUSIONS": 9
  }
  return [dow_with_BIO[x] for x in seq_label]


In [ ]:
df_three_window_train["labels_code"] = df_three_window_train["labels_BIO"].apply(rep_BIO)
df_three_window_test["labels_code"] = df_three_window_test["labels_BIO"].apply(rep_BIO)
df_three_window_test_structured["labels_code"] = df_three_window_test_structured["labels_BIO"].apply(rep_BIO)
df_three_window_test_unstructured["labels_code"] = df_three_window_test_unstructured["labels_BIO"].apply(rep_BIO)
df_three_window_test_pubmed["labels_code"] = df_three_window_test_pubmed["labels_BIO"].apply(rep_BIO)

In [ ]:
# tokenized_train = Dataset.from_pandas(convert(train_df[:4000]))
# tokenized_eval = Dataset.from_pandas(convert(eval_df[:1000]))
tokenized_train_manually = Dataset.from_pandas(convert(df_three_window_train))
tokenized_eval_manually = Dataset.from_pandas(convert(df_three_window_test))
tokenized_eval_manually_structured = Dataset.from_pandas(convert(df_three_window_test_structured))
tokenized_eval_manually_unstructured = Dataset.from_pandas(convert(df_three_window_test_unstructured))
tokenized_eval_pubmed = Dataset.from_pandas(convert(df_three_window_test_pubmed))
tokenized_train_manually.set_format("torch")
tokenized_eval_manually.set_format("torch")
tokenized_eval_manually_structured.set_format("torch")
tokenized_eval_manually_unstructured.set_format("torch")
tokenized_eval_pubmed.set_format("torch")
train_loader_manually = DataLoader(tokenized_train_manually, shuffle=False, batch_size=4)
eval_loader_manually = DataLoader(tokenized_eval_manually, shuffle=False, batch_size=4)
eval_loader_manually_structured = DataLoader(tokenized_eval_manually_structured, shuffle=False, batch_size=4)
eval_loader_manually_unstructured = DataLoader(tokenized_eval_manually_unstructured, shuffle=False, batch_size=4)
eval_loader_pubmed = DataLoader(tokenized_eval_pubmed, shuffle=False, batch_size=4)

In [ ]:
def F1(ref, pred):
    tp = fn = fp = 0
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            if pred[i][j] == 1 and ref[i][j] == 1:
                tp += 1
            elif pred[i][j] == 1 and ref[i][j] == 0:
                fp += 1
            elif pred[i][j] == 0 and ref[i][j] == 1:
                fn += 1
    pc = rc = f1 = 0
    if tp != 0:
        pc = tp/(tp+fp)
        rc = tp/(tp+fn)
        f1 = 2*pc*rc/(pc+rc)
    print(f'precision: {pc}, recall: {rc}, F1: {f1}')
    return f1


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

# adapted from https://github.com/DHPO/GHM_Loss.pytorch
class GHM_Loss(nn.Module):
    def __init__(self, bins, alpha, ignore_label_id=-100):
        super(GHM_Loss, self).__init__()
        self._bins = bins
        self._alpha = alpha
        self._last_bin_count = None
        self._ignore_label_id = ignore_label_id

    def _g2bin(self, g):
        return torch.floor(g * (self._bins - 0.0001)).long()

    def _custom_loss(self, x, target, weight):
        raise NotImplementedError

    def _custom_loss_grad(self, x, target):
        raise NotImplementedError

    def forward(self, x, target):
        valid_label = (target != self._ignore_label_id)
        x = x[valid_label]
        target = target[valid_label]

        g = torch.abs(self._custom_loss_grad(x, target)).detach()
        bin_idx = self._g2bin(g)
        bin_count = torch.zeros((self._bins), device=x.device)
        for i in range(self._bins):
            bin_count[i] = (bin_idx == i).sum().item()
        N = x.size(0)  # N = (x.size(0) * x.size(1))
        if self._last_bin_count is None:
            self._last_bin_count = bin_count
        else:
            bin_count = self._alpha * self._last_bin_count + \
                (1 - self._alpha) * bin_count
            self._last_bin_count = bin_count
        nonempty_bins = (bin_count > 0).sum().item()
        gd = bin_count * nonempty_bins
        gd = torch.clamp(gd, min=1)  # min=0.0001
        beta = N / gd
        return self._custom_loss(x, target, beta[bin_idx])


class GHMC_Loss(GHM_Loss):
    def __init__(self, bins, alpha):
        super(GHMC_Loss, self).__init__(bins, alpha)

    def _custom_loss(self, x, target, weight):
        criterion = CrossEntropyLoss(reduction='none')
        loss = criterion(x, target)
        loss = torch.mean(loss * weight)
        return loss

    def _custom_loss_grad(self, x, target):
        return torch.gather(F.softmax(x, dim=-1).detach(), 1, target.view(-1, 1)) - 1.0

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix as mcm, classification_report

def evaluate(model, data_loader, criterion, device=device):
    model.eval()
    val_true, val_pred = [], []
    all_true, all_pred = [], []
    i = 0
    eval_loss = 0
    for batch in data_loader:
        i += 1
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
            eval_loss += criterion(logits.view(-1, 10),
                                   batch['labels'].view(-1)).item()
        y_pred = torch.argmax(logits, dim=-1).detach().cpu().numpy().tolist()
        y_true = batch['labels'].detach().cpu().numpy().tolist()  # .squeeze()
        real_len = torch.sum(batch['attention_mask'],
                             1).detach().cpu().numpy().tolist()
        for j in range(len(real_len)):
            pred_tmp = []
            true_tmp = []
            for k in range(real_len[j]):  # range(len(y_true[j])):
                if y_true[j][k] != ignore_label_id:
                    pred_tmp.append(y_pred[j][k])
                    true_tmp.append(y_true[j][k])
            val_true.append(true_tmp)
            val_pred.append(pred_tmp)
            all_true.extend(true_tmp)
            all_pred.extend(pred_tmp)
    print("true: ", val_true)
    print("pred: ", val_pred)
    eval_loss = eval_loss / (i+1)
    f1 = F1(val_true, val_pred)
    confusion_matrix_eval = confusion_matrix(all_true, all_pred)
    target_names =  ["B_BACKGROUND", "I_BACKGROUND", "B_OBJECTIVE", "I_OBJECTIVE", "B_METHODS", "I_METHODS", "B_RESULTS", "I_RESULTS", "B_CONCLUSIONS", "I_CONCLUSIONS"]
    classificaiton_report_eval = classification_report(all_true, all_pred, target_names = target_names)



    return f1, eval_loss, confusion_matrix_eval, classificaiton_report_eval


In [ ]:
def train_and_eval(model, train_loader, valid_loader, criterion,
                   optimizer, scheduler, num_epochs, device=device):
    best_acc = 0.0
    # patience = 0
    progress_bar = tqdm(range(num_training_steps))
    for epoch in range(num_epochs):
        count = 0
        model.train()
        print("***** Running training epoch {} *****".format(epoch+1))
        running_loss = 0
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            loss = criterion(logits.view(-1, 10), batch['labels'].view(-1))
            loss /= grad_acu_steps
            running_loss += loss.item()
            loss.backward()
            count += 1
            if count % grad_acu_steps == 0:
                if count % (grad_acu_steps*20) == 0:
                    running_loss /= 20
                    print(f'running_loss:{running_loss}')
                    running_loss = 0
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
        model.eval()
        acc, eval_loss, confusion_matrix_eval, classificaiton_report_eval = evaluate(model, valid_loader, criterion)
        print(f'evaluation loss: {eval_loss}')
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_model.pth")
        print("current acc is {:.4f}, best acc is {:.4f}".format(acc, best_acc))

In [ ]:
grad_acu_steps = 4 
num_epochs = 5
num_training_steps = num_epochs*len(train_loader_manually)//grad_acu_steps

config = BertConfig.from_pretrained(
    model_path, max_position_embeddings=maxlen, num_labels=10)  # 18
model = BertForTokenClassification.from_pretrained(
    model_path, config=config, ignore_mismatched_sizes=True).to(device)
#optimizer = AdamW([{'params': model.bert.parameters()},
                   #{'params': model.classifier.parameters(), 'lr': 5e-5}], lr=1e-5)

no_decay = ['bias', 'LayerNorm']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.bert.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.bert.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
{'params': [p for n, p in model.classifier.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01, 'lr': 1e-4},
{'params': [p for n, p in model.classifier.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0, 'lr': 1e-4},
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=200,
                                            num_training_steps=num_training_steps, num_cycles=0.5)
criterion = GHMC_Loss(bins=10, alpha=0.9)

train_and_eval(model, train_loader_manually, eval_loader_manually,
               criterion, optimizer, scheduler, num_epochs)

Some weights of the model checkpoint at /content/gdrive/My Drive/scibert were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

  0%|          | 0/563 [00:00<?, ?it/s]

***** Running training epoch 1 *****
running_loss:2.2057857900857925
running_loss:2.257635323703289
running_loss:1.6914961449801922
running_loss:1.4423181988298892
running_loss:1.3809612385928631
true:  [[0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 4], [0, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running_loss:1.3356353960931302
running_loss:1.2814112439751626
running_loss:1.1301460929214955
running_loss:0.9544509526342153
running_loss:1.0747656732797624
true:  [[0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 4], [0, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 

...

current acc is 0.9931, best acc is 0.9931
***** Running training epoch 4 *****
running_loss:0.5159475192776881
running_loss:0.4388108258368447
running_loss:0.4189555502845906
running_loss:0.37974651046679353
running_loss:0.6106036140583455
true:  [[0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 4], [0, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


current acc is 0.9962, best acc is 0.9962
***** Running training epoch 5 *****
running_loss:0.30948053037864154
running_loss:0.2871857553196605
running_loss:0.21126942476839758
running_loss:0.24697397183044814
running_loss:0.465782808768563
true:  [[0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 4], [0, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4,

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


current acc is 0.9963, best acc is 0.9963


In [ ]:
 f1, eval_loss, confusion_matrix_eval, classificaiton_report_eval = evaluate(model, eval_loader_manually, criterion)
 

true:  [[0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 4], [0, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [8, 9, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 1, 2], [0, 2, 4], [2, 4, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 8], [6, 8, 9], [0, 2, 4], [2, 4, 5], [4, 5, 5], [4, 5, 6], [4, 6, 7], [6, 7, 7], [6, 7, 7], [6

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.eval()
val_true, val_pred = [], []
all_true, all_pred = [], []
i = 0
eval_loss = 0
for batch in eval_loader_manually:
    i += 1
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        eval_loss += criterion(logits.view(-1, 10),
                                batch['labels'].view(-1)).item()
    y_pred = torch.argmax(logits, dim=-1).detach().cpu().numpy().tolist()
    y_true = batch['labels'].detach().cpu().numpy().tolist()  # .squeeze()
    real_len = torch.sum(batch['attention_mask'],
                          1).detach().cpu().numpy().tolist()
    for j in range(len(real_len)):
        pred_tmp = []
        true_tmp = []
        for k in range(real_len[j]):  # range(len(y_true[j])):
            if y_true[j][k] != ignore_label_id:
                pred_tmp.append(y_pred[j][k])
                true_tmp.append(y_true[j][k])
        val_true.append(true_tmp)
        val_pred.append(pred_tmp)
        all_true.extend(true_tmp)
        all_pred.extend(pred_tmp)

In [ ]:
len(val_pred)

875

In [ ]:
def back_to_single(pred, true, l1):
  return_l_true = []
  return_l_pred = []
  marker = 0
  for i in l1: 
    new_l_true = []
    new_l_pred = []
    add = len(i) - 2
    pred_single = pred[marker : marker + add]
    true_single = true[marker : marker + add]
    marker = marker + add
    for j in range(len(pred_single)):
      new_l_pred.append(pred_single[j][0])
      new_l_true.append(true_single[j][0])
    new_l_pred.append(pred_single[-1][1])
    new_l_pred.append(pred_single[-1][2])
    new_l_true.append(true_single[-1][1])
    new_l_true.append(true_single[-1][2])
    return_l_true.append(new_l_true)
    return_l_pred.append(new_l_pred)
  return return_l_pred, return_l_true

def map_to_normal(seq_label):
  normal_map = {
    "0": "B",
    "1": "B",
    "2": "O",
    "3": "O",
    "4": "M",
    "5": "M",
    "6": "R", 
    "7": "R",
    "8": "C",
    "9": "C"
  }
  return [[normal_map[str(y)] for y in x ] for x in seq_label]


*test* on whole testing set

In [ ]:
pred_single, true_single = back_to_single(val_pred, val_true, test_y)

In [ ]:
pred_single_normal = map_to_normal(pred_single)
true_single_normal = map_to_normal(true_single)
pred_final = [j for i in pred_single_normal for j in i ]
true_final = [j for i in true_single_normal for j in i ]

In [ ]:
cm_labels = [ "B","C","M","O", "R"]
print(classification_report(true_final, pred_final,target_names=cm_labels))

              precision    recall  f1-score   support

           B       0.97      0.91      0.94       186
           C       0.92      0.81      0.86       161
           M       0.93      0.93      0.93       271
           O       0.96      0.96      0.96        95
           R       0.87      0.93      0.90       362

    accuracy                           0.91      1075
   macro avg       0.93      0.91      0.92      1075
weighted avg       0.91      0.91      0.91      1075



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_final, pred_final)

array([[169,   0,   9,   0,   8],
       [  2, 130,   0,   0,  29],
       [  0,   0, 253,   3,  15],
       [  1,   0,   3,  91,   0],
       [  3,  12,   8,   1, 338]])

*test* on structured testing set

In [ ]:
model.eval()
val_true_structured, val_pred_structured = [], []
all_true_structured, all_pred_structured = [], []
i = 0
eval_loss = 0
for batch in eval_loader_manually_structured:
    i += 1
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        eval_loss += criterion(logits.view(-1, 10),
                                batch['labels'].view(-1)).item()
    y_pred = torch.argmax(logits, dim=-1).detach().cpu().numpy().tolist()
    y_true = batch['labels'].detach().cpu().numpy().tolist()  # .squeeze()
    real_len = torch.sum(batch['attention_mask'],
                          1).detach().cpu().numpy().tolist()
    for j in range(len(real_len)):
        pred_tmp = []
        true_tmp = []
        for k in range(real_len[j]):  # range(len(y_true[j])):
            if y_true[j][k] != ignore_label_id:
                pred_tmp.append(y_pred[j][k])
                true_tmp.append(y_true[j][k])
        val_true_structured.append(true_tmp)
        val_pred_structured.append(pred_tmp)
        all_true_structured.extend(true_tmp)
        all_pred_structured.extend(pred_tmp)

In [ ]:
pred_single, true_single = back_to_single(val_pred_structured, val_true_structured, test_y_structured)
pred_single_normal = map_to_normal(pred_single)
true_single_normal = map_to_normal(true_single)
pred_final = [j for i in pred_single_normal for j in i ]
true_final = [j for i in true_single_normal for j in i ]
cm_labels = [ "B","C","M","O", "R"]
print(classification_report(true_final, pred_final,target_names=cm_labels))

              precision    recall  f1-score   support

           B       0.95      0.91      0.93       116
           C       0.95      0.83      0.89       127
           M       0.97      0.94      0.95       235
           O       0.97      0.97      0.97        69
           R       0.87      0.97      0.92       262

    accuracy                           0.93       809
   macro avg       0.94      0.92      0.93       809
weighted avg       0.93      0.93      0.93       809



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_final, pred_final)

array([[106,   0,   5,   0,   5],
       [  2, 105,   0,   0,  20],
       [  0,   0, 221,   2,  12],
       [  1,   0,   1,  67,   0],
       [  2,   5,   2,   0, 253]])

test on unstructured testing set 

In [ ]:
model.eval()
val_true_unstructured, val_pred_unstructured = [], []
all_true_unstructured, all_pred_unstructured = [], []
i = 0
eval_loss = 0
for batch in eval_loader_manually_unstructured:
    i += 1
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        eval_loss += criterion(logits.view(-1, 10),
                                batch['labels'].view(-1)).item()
    y_pred = torch.argmax(logits, dim=-1).detach().cpu().numpy().tolist()
    y_true = batch['labels'].detach().cpu().numpy().tolist()  # .squeeze()
    real_len = torch.sum(batch['attention_mask'],
                          1).detach().cpu().numpy().tolist()
    for j in range(len(real_len)):
        pred_tmp = []
        true_tmp = []
        for k in range(real_len[j]):  # range(len(y_true[j])):
            if y_true[j][k] != ignore_label_id:
                pred_tmp.append(y_pred[j][k])
                true_tmp.append(y_true[j][k])
        val_true_unstructured.append(true_tmp)
        val_pred_unstructured.append(pred_tmp)
        all_true_unstructured.extend(true_tmp)
        all_pred_unstructured.extend(pred_tmp)

In [ ]:
pred_single, true_single = back_to_single(val_pred_unstructured, val_true_unstructured, test_y_unstructured)
pred_single_normal = map_to_normal(pred_single)
true_single_normal = map_to_normal(true_single)
pred_final = [j for i in pred_single_normal for j in i ]
true_final = [j for i in true_single_normal for j in i ]
cm_labels = [ "B","C","M","O", "R"]
print(classification_report(true_final, pred_final,target_names=cm_labels))

              precision    recall  f1-score   support

           B       0.98      0.90      0.94        70
           C       0.78      0.74      0.76        34
           M       0.73      0.89      0.80        36
           O       0.92      0.92      0.92        26
           R       0.85      0.85      0.85       100

    accuracy                           0.86       266
   macro avg       0.85      0.86      0.85       266
weighted avg       0.87      0.86      0.86       266



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_final, pred_final)

array([[63,  0,  4,  0,  3],
       [ 0, 25,  0,  0,  9],
       [ 0,  0, 32,  1,  3],
       [ 0,  0,  2, 24,  0],
       [ 1,  7,  6,  1, 85]])

*test* on PubMed 20k RCT

In [ ]:
model.eval()
val_true_pubmed, val_pred_pubmed = [], []
all_true_pubmed, all_pred_pubmed = [], []
i = 0
eval_loss = 0
for batch in eval_loader_pubmed:
    i += 1
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        eval_loss += criterion(logits.view(-1, 10),
                                batch['labels'].view(-1)).item()
    y_pred = torch.argmax(logits, dim=-1).detach().cpu().numpy().tolist()
    y_true = batch['labels'].detach().cpu().numpy().tolist()  # .squeeze()
    real_len = torch.sum(batch['attention_mask'],
                          1).detach().cpu().numpy().tolist()
    for j in range(len(real_len)):
        pred_tmp = []
        true_tmp = []
        for k in range(real_len[j]):  # range(len(y_true[j])):
            if y_true[j][k] != ignore_label_id:
                pred_tmp.append(y_pred[j][k])
                true_tmp.append(y_true[j][k])
        val_true_pubmed.append(true_tmp)
        val_pred_pubmed.append(pred_tmp)
        all_true_pubmed.extend(true_tmp)
        all_pred_pubmed.extend(pred_tmp)

In [ ]:
pred_single, true_single = back_to_single(val_pred_pubmed, val_true_pubmed, test_y_pubmed)
pred_single_normal = map_to_normal(pred_single)
true_single_normal = map_to_normal(true_single)
pred_final = [j for i in pred_single_normal for j in i ]
true_final = [j for i in true_single_normal for j in i ]
cm_labels = [ "B","C","M","O", "R"]
print(classification_report(true_final, pred_final,target_names=cm_labels))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_final, pred_final)

In [ ]:
df_three_window_train[:10]
